# <font color="red">**Assignment 8 Analisi e Classificazione delle Email per la Rilevazione di SPAM**</font>
ProfessionAI, azienda specializzata nell'automazione basata sull'Intelligenza Artificiale, vuole sviluppare una libreria software in grado di analizzare e classificare le email ricevute. L'obiettivo principale è identificare le email di tipo SPAM per condurre successivamente delle analisi approfondite sui contenuti.

Il CEO ha espresso l'esigenza di focalizzarsi su email SPAM per comprendere meglio le tendenze, i contenuti e i comportamenti associati. Queste informazioni verranno utilizzate per migliorare la sicurezza delle comunicazioni aziendali e perfezionare i filtri anti-spam.

# <font color="red">**Obiettivo del Progetto**</font>
Il CTO ha fornito un dataset di email per realizzare le seguenti attività:
 1. **Addestrare un classificatore** per identificare le email SPAM.
 2. **Individuare i Topic principali** tra le email classificate come SPAM.
 3. **Calcolare la distanza semantica** tra i topics ottenuti per valutare l'eterogeneità dei contenuti delle email SPAM.
 4. **Estrarre dalle email NON SPAM** le informazioni sulle Organizzazioni menzionate.


# <font color="red">**Valore Aggiunto**</font>
L'analisi delle email permette a ProfessionAI di ottenere diversi vantaggi strategici:
- **Miglioramento del filtro anti-spam**: Un classificatore efficiente permette di ridurre significativamente il volume di email indesiderate che raggiungono la casella di posta, ottimizzando la gestione delle comunicazioni aziendali.
- **Analisi contenutistica approfondita**: L'individuazione dei principali topic trattati nelle email SPAM consente di ottenere informazioni preziose sui trend, tematiche e schemi ricorrenti, potenziando le strategie di cybersecurity.
- **Valutazione dell'eterogeneità**: La distanza semantica tra i topic consente di comprendere la diversità dei contenuti SPAM, utile per ottimizzare le difese contro un'ampia gamma di attacchi.
- **Identificazione di organizzazioni**: L'estrazione di organizzazioni dalle email legittime può essere sfruttata per migliorare i processi di business intelligence e gestire meglio le comunicazioni con clienti e partner.

**Installazione pacchetti**

In [ ]:
!pip install gensim

In [ ]:
# Import delle librerie necessarie
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np
from scipy.spatial.distance import cosine
import spacy
import nltk
from nltk.corpus import stopwords
from gensim import corpora, models
import string

### <font color="yellow">***Caricamento Dataset & Data Cleaning***</font>

In [ ]:
# caricamento dataset
url = ("https://raw.githubusercontent.com/ProfAI/natural-language-processing/"
       "main/datasets/Verifica%20Finale%20-%20Spam%20Detection/spam_dataset.csv")

df = pd.read_csv(url)
display(df)

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\nth...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\n( see a...",0
2,3624,ham,"Subject: neon retreat\nho ho ho , we ' re arou...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\nthis deal is to ...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\nthe transport v...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\nhpl ...,0
5168,2933,ham,Subject: calpine daily gas nomination\n>\n>\nj...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


In [ ]:
# vedo alcuni esempi di testo e la relativa classificazione
print(df['text'].iloc[0],'\n\n',df['label'].iloc[0],  '\n\n')
print(df['text'].iloc[10],'\n\n',df['label'].iloc[10], '\n\n')
print(df['text'].iloc[100],'\n\n',df['label'].iloc[100])

Subject: enron methanol ; meter # : 988291
this is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminary
flow data provided by daren } .
please override pop ' s daily volume { presently zero } to reflect daily
activity you can obtain from gas control .
this change is needed asap for economics purposes . 

 ham 


Subject: vocable % rnd - word asceticism
vcsc - brand new stock for your attention
vocalscape inc - the stock symbol is : vcsc
vcsc will be our top stock pick for the month of april - stock expected to
bounce to 12 cents level
the stock hit its all time low and will bounce back
stock is going to explode in next 5 days - watch it soar
watch the stock go crazy this and next week .
breaking news - vocalscape inc . announces agreement to resell mix network
services
current price : $ 0 . 025
we expect projected speculative price in next 5 days : $ 0 . 12
we expect projected speculative price in next 15 days : $ 0 . 15
vocalscape networks inc . is building a compan

Il dataset consiste in 4 colonne, la prima (senza nome) che rappresenta una sorta di indicizzazione delle righe, poi abbiamo la label che ci dice se il contenuto della mail è spam oppure no, il testo della mail, con anche il soggetto, e infine la colonna label ma in formato binario anziché stringa

In [ ]:
print("Suddivisioni classi nel dataset di partenza")
print("Percentuale di eMail classificate come Spam: ", round(len(df[df.label == "spam"])/len(df),2), "%")
print("Percentuale di eMail classificate come Non Spam: ", round(len(df[df.label == "ham"])/len(df),2), "%")

Suddivisioni classi nel dataset di partenza
Percentuale di eMail classificate come Spam:  0.29 %
Percentuale di eMail classificate come Non Spam:  0.71 %


Il dataset non è perfettamente bilanciato, però comunque il divario tra le 2 classi non risulta cosi accentuato (la classe minoritaria è ben rappresentata lo stesso).

Posso procedere senza dover andare ad applicare undersampling/oversampling.

DATA CLEANING

In [ ]:
#scarico le stopwords se non sono presenti
try:
    _ = stopwords.words("english")
except LookupError:
    nltk.download("stopwords", quiet=True)

english_stopwords = set(stopwords.words("english"))  #uso il set per togliere eventuali duplicati
nlp = spacy.load("en_core_web_sm")

punctuation_table = str.maketrans({c: " " for c in string.punctuation})

# uso la funzione per pulire il testo
def data_cleaner(sentence: str) -> str:
    sentence = sentence.lower()
    sentence = sentence.translate(punctuation_table) #uso maketrans+translate al posto del for all'interno della funzione
    doc = nlp(sentence)
    lemmas = [tok.lemma_ for tok in doc if tok.is_alpha]
    tokens = [tok for tok in lemmas if tok not in english_stopwords and len(tok) > 1]  # applico anche un filtro sulla lunghezza delle parole in quanti facendo alcuni
                                                                                       # test mi sono trovato poi all'interno del dizionario lettere singole senza un senso preciso
    tokens = [re.sub(r"\d", "", tok) for tok in tokens if tok]
    return " ".join(tokens)

In [ ]:
#uso il TfidfVectorizer per dare i pesi a ciascun elemento delle varie parole contenute nelle mail
vectorizer = TfidfVectorizer(
    preprocessor=data_cleaner,
    tokenizer=str.split, #applico la tokenizzazione splittando le varie parole
    token_pattern=None, #qui metto token pattern = None visto che ho già effettuato il cleaning usando la funzione data_cleaner
    min_df=3, #scarto i termini rari che appaiono in meno di 3 documenti, riduco il rumore e abbasso la dimensionalità
    max_df=0.3 #scarto i termini che appaiono in più del 30% dei documenti (anche qui riduco la dimensionalità e tolgo i termini che appaiono molto spesso e quindi non sono informativi: tipo la parola "subject")
)

X = vectorizer.fit_transform(df["text"].values)
y = df["label"].map({"ham": 0, "spam": 1}).values

In [ ]:
print(y)

[0 0 0 ... 0 0 1]


In [ ]:
print(X)

  (0, 3996)	0.09412193784134303
  (0, 6716)	0.23213511240934784
  (0, 6714)	0.1175491885550849
  (0, 4517)	0.12857747211720508
  (0, 7182)	0.15568808871249135
  (0, 4823)	0.144003705012492
  (0, 6880)	0.17311970842293198
  (0, 7981)	0.24281360198142626
  (0, 4495)	0.13844934838134543
  (0, 3043)	0.1944947698362905
  (0, 8149)	0.15421551417847493
  (0, 3023)	0.10867883077443628
  (0, 7454)	0.3058874198092447
  (0, 7895)	0.19644493414680173
  (0, 2995)	0.306053958851577
  (0, 10532)	0.11943907503643691
  (0, 8004)	0.232927438962433
  (0, 10918)	0.19232943932081387
  (0, 8425)	0.19779776513257435
  (0, 125)	0.181742043979165
  (0, 7244)	0.22910862257805456
  (0, 4715)	0.1046080905482578
  (0, 2656)	0.18292719213414182
  (0, 1994)	0.12490394604035944
  (0, 7058)	0.1026082465050001
  :	:
  (5170, 8059)	0.05440494103431993
  (5170, 3083)	0.06886906593097539
  (5170, 1000)	0.2151701572477338
  (5170, 8138)	0.0789394054667922
  (5170, 1002)	0.45037639075983366
  (5170, 1135)	0.0780198985472324

Dopo aver creato il dizionario e trasformato tutto il corpus documentale in formato numerico posso procedere con la creazione del modello di classificazione.
In questo caso utilizzerò una regressione logistica

### <font color="yellow">***Classificatore eMail spam***</font>

In [ ]:
#applico il solito train test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
#creo il modello di regressione
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred, target_names=["HAM", "SPAM"]))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))


=== Classification Report ===
              precision    recall  f1-score   support

         HAM       0.99      0.98      0.99       735
        SPAM       0.95      0.99      0.97       300

    accuracy                           0.98      1035
   macro avg       0.97      0.98      0.98      1035
weighted avg       0.98      0.98      0.98      1035

Confusion Matrix:
 [[721  14]
 [  4 296]]
Accuracy: 0.9826086956521739


Con questo semplice modello di regressione logistica siamo stati in grado di ottenere un classificatore molto buono, in particolare abbiamo una accuracy complessiva di più del 98% (OTTIMO!!)

### <font color="yellow">***Individuazione topics eMail spam***</font>

In [ ]:
NUM_TOPICS = 10    #decido aribtrariamente di individuare 10 argomenti prinicipali

#copio il dataframe (filtrando solo le righe spam), pulisco e applico la tokenizzazione
spam_df = df[df.label == "spam"].copy()
spam_clean = spam_df.text.apply(data_cleaner)
spam_tokens = [doc.split() for doc in spam_clean]


dictionary = corpora.Dictionary(spam_tokens)

# vedo la percentuale con cui si presentano i token più ricorrenti, per capire poi quale soglia utilizzare per l'eliminazione dei token più frequenti
if True:
    import pandas as pd
    num_docs = len(spam_tokens)
    df_series  = pd.Series({dictionary[id]: cnt for id, cnt in dictionary.dfs.items()})
    pct_series = (df_series / num_docs).sort_values(ascending=False)

    print("\nTop 25 tokens per frequenza nei documenti (prima del filtering):")
    for token, pct in pct_series.head(25).items():
        print(f"{token:15s}  {df_series[token]:4d} docs  ({pct*100:5.1f}%)")


Top 25 tokens per frequenza nei documenti (prima del filtering):
subject          1499 docs  (100.0%)
get               478 docs  ( 31.9%)
http              475 docs  ( 31.7%)
com               444 docs  ( 29.6%)
please            313 docs  ( 20.9%)
time              297 docs  ( 19.8%)
email             296 docs  ( 19.7%)
price             295 docs  ( 19.7%)
good              281 docs  ( 18.7%)
one               280 docs  ( 18.7%)
www               263 docs  ( 17.5%)
need              253 docs  ( 16.9%)
new               253 docs  ( 16.9%)
offer             251 docs  ( 16.7%)
use               234 docs  ( 15.6%)
go                231 docs  ( 15.4%)
want              225 docs  ( 15.0%)
take              221 docs  ( 14.7%)
information       220 docs  ( 14.7%)
click             216 docs  ( 14.4%)
free              211 docs  ( 14.1%)
like              208 docs  ( 13.9%)
product           208 docs  ( 13.9%)
make              203 docs  ( 13.5%)
send              203 docs  ( 13.5%)


In [ ]:
#anche qui faccio un filtro su termini contenuti all'interno del dizionario, togliendo quelli troppo rari o eccessivamente presenti
NO_BELOW = 3
NO_ABOVE = 0.25 #applico 0.25 per togliere alcuni termini poco utili com http, com, get

print(f"Appicando filter_extremes(no_below={NO_BELOW}, no_above={NO_ABOVE})…")

dictionary.filter_extremes(no_below=NO_BELOW, no_above=NO_ABOVE)

corpus = [dictionary.doc2bow(tokens) for tokens in spam_tokens]

lda = models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=NUM_TOPICS,
    passes=10,
    random_state=42,
)

print("\n=== LDA Topics (SPAM) ===")
for i, topic in lda.print_topics(num_words=10):
    print(f"Topic {i}: {topic}")

Appicando filter_extremes(no_below=3, no_above=0.25)…

=== LDA Topics (SPAM) ===
Topic 0: 0.009*"say" + 0.006*"one" + 0.005*"gas" + 0.004*"source" + 0.004*"company" + 0.004*"story" + 0.004*"price" + 0.004*"project" + 0.004*"use" + 0.004*"full"
Topic 1: 0.008*"online" + 0.006*"man" + 0.005*"well" + 0.005*"good" + 0.005*"med" + 0.005*"order" + 0.005*"time" + 0.005*"need" + 0.004*"account" + 0.004*"make"
Topic 2: 0.014*"cialis" + 0.012*"viagra" + 0.012*"soft" + 0.010*"drug" + 0.009*"tab" + 0.009*"prescription" + 0.007*"hour" + 0.007*"new" + 0.006*"product" + 0.006*"pill"
Topic 3: 0.048*"font" + 0.047*"td" + 0.041*"nbsp" + 0.035*"height" + 0.030*"width" + 0.024*"size" + 0.022*"align" + 0.021*"tr" + 0.019*"border" + 0.018*"color"
Topic 4: 0.011*"computron" + 0.010*"please" + 0.010*"contact" + 0.010*"www" + 0.008*"free" + 0.008*"remove" + 0.008*"message" + 0.008*"mail" + 0.007*"email" + 0.007*"send"
Topic 5: 0.035*"pill" + 0.012*"cd" + 0.011*"mg" + 0.011*"price" + 0.009*"paliourg" + 0.009*"m

10 argomenti forse sono un po' tanti. Provo a ridurre il numero da 10 a 5

In [ ]:
NUM_TOPICS = 5
lda = models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=NUM_TOPICS,
    passes=10,
    random_state=42,
)

print("\n=== LDA Topics (SPAM) ===")
for i, topic in lda.print_topics(num_words=10):
    print(f"Topic {i}: {topic}")


=== LDA Topics (SPAM) ===
Topic 0: 0.020*"company" + 0.012*"statement" + 0.010*"stock" + 0.008*"information" + 0.007*"may" + 0.007*"investment" + 0.007*"security" + 0.007*"report" + 0.006*"within" + 0.006*"price"
Topic 1: 0.009*"price" + 0.005*"good" + 0.005*"online" + 0.005*"save" + 0.005*"software" + 0.005*"adobe" + 0.005*"need" + 0.004*"microsoft" + 0.004*"want" + 0.004*"well"
Topic 2: 0.015*"pill" + 0.008*"viagra" + 0.007*"cialis" + 0.005*"prescription" + 0.005*"soft" + 0.005*"drug" + 0.004*"mg" + 0.004*"tab" + 0.003*"good" + 0.003*"hour"
Topic 3: 0.032*"font" + 0.031*"td" + 0.026*"nbsp" + 0.023*"height" + 0.019*"width" + 0.016*"size" + 0.014*"align" + 0.014*"tr" + 0.012*"color" + 0.012*"border"
Topic 4: 0.010*"please" + 0.009*"www" + 0.008*"contact" + 0.008*"computron" + 0.007*"free" + 0.007*"email" + 0.007*"remove" + 0.006*"message" + 0.006*"send" + 0.006*"mail"


Ok direi che qui abbiamo chiaramente individuato qualche tipologia ricorrente di argomenti presenti nella email Spam:
- Topic 0: sembra essere riferito all'ambito investimenti (magari pubblicità di qualche broker finanziario)
- Topic 1: qui invece vendita (probabilmente truffaldina) di software per il PC
- Topic 2: questo invece riguarda la vendita di prodotti legati al mondo del sesso
- Topic 3: non sembrano avere un significato preciso, probabilmente mail con argomenti vari sono stati accorpate assieme
- Topic 4: anche qui l'argomento non è chiaro, ma probabilmente sono mail che invitano a cliccare/visitare qualche sito (probabilmente malevolo)

### <font color="yellow">***Distanza semantica tra gli argomenti delle mail SPAM***</font>

Qui di seguito mi creo una matrice per andare a vedere coppia a coppia la distanza semantica tra un argomento e l'altro. Distanza sempre calcolata utilizzando la cosine similiraty

In [ ]:
topic_vecs = np.zeros((NUM_TOPICS, len(dictionary)))
for t in range(NUM_TOPICS):
    for wid, weight in lda.get_topic_terms(t, topn=len(dictionary)):
        topic_vecs[t, wid] = weight

dist_matrix = np.zeros((NUM_TOPICS, NUM_TOPICS))
for i in range(NUM_TOPICS):
    for j in range(i + 1, NUM_TOPICS):
        d = cosine(topic_vecs[i], topic_vecs[j])
        dist_matrix[i, j] = dist_matrix[j, i] = d

print(dist_matrix)

mean_dist = dist_matrix[np.triu_indices(NUM_TOPICS, k=1)].mean() #prendo solo la parte di matrice triangolare superiore per il calcolo della media (escludendo anche la diagonale che tanto è sempre 0)
print(f"\nDistanza media fra i vari argomenti: {mean_dist:.4f}")

[[0.         0.64314482 0.81040995 0.92186581 0.66537708]
 [0.64314482 0.         0.57014181 0.84959885 0.51464942]
 [0.81040995 0.57014181 0.         0.90413364 0.72849621]
 [0.92186581 0.84959885 0.90413364 0.         0.82895915]
 [0.66537708 0.51464942 0.72849621 0.82895915 0.        ]]

Distanza media fra i vari argomenti: 0.7437


Possiamo notare come gli argomenti più simili siano il Topic 1, con il topic 4. (anche se non trovo un significato a questo)

Inoltre topic 1 e 2 sono abbastanza simili, in quanto riguardano entrambi una vendita online.

In generale la distanza media fra i vari argomenti è 0.74, quindi abbastanza alta, le mail SPAM posso essere quindi di tipologia abbastanza varia.

### <font color="yellow">***Identificazione organizzazioni***</font>

Nel codice seguente scarico prima di tutto il dizionario delle entità da spacy (e da questo filtro solo le entità segnate con "ORG"), e successivamente vado a vedere nelle mail lecite quali sono le organizzazioni più citate.

In [ ]:
NLP_NER = spacy.load("en_core_web_sm")


def extract_orgs(text: str):
    return [ent.text for ent in NLP_NER(text).ents if ent.label_ == "ORG"]

ham_df = df[df.label == "ham"].copy()
ham_df["orgs"] = ham_df.text.apply(extract_orgs)

all_orgs = [org for sublist in ham_df.orgs for org in sublist]
org_freq = pd.Series(all_orgs).value_counts()
print("\n=== Top 20 Organisations in HAM ===")
print(org_freq.head(20))



=== Top 20 Organisations in HAM ===
                                       271
doc                                     178
north america corp .                    166
ami chokshi / corp / enron              153
xls                                     123
enron north america corp .               81
pg & e                                   71
riley / hou /                            57
mary m smith / hou /                     53
exxon                                    52
enron corp .                             51
boas / hou                               45
coastal oil & gas corporation\n          43
lamadrid / hou                           42
capital & trade resources corp .         39
texaco                                   38
ami chokshi / corp / enron @ enron\n     38
stella l morris / hou                    38
d & h gas company                        37
zivley / hou                             36
Name: count, dtype: int64


Andando a cercare in internet sembrano essere tutte compagnie legate al mondo Energetico (Vendita di gas naturale, petrolio, energia elettrica in generale e il suo smistamento).

L'area geografica di provenienza è il nord america (Stati Uniti in particolare)

Quindi le mail lecite probabilmente provengono da qualche azienda (forse di consulenza) che fa consulenza per compagnie energetiche o magari qualche società che collabora con esse (tipo per la costruzione delle infrastrutture).